In [1]:
import pandas as pd
patient = pd.read_csv('PatientRoute.csv')
patient = patient[(patient['province']=='Seoul')]
patient = patient.drop_duplicates()

In [2]:
patient['date'] = pd.to_datetime(patient['date'])

In [3]:
def last_loc(patient):
    route = []
    for pat in patient['patient_id'].unique():
        uni_patient = patient[patient['patient_id'] == pat]
        uni_patient = uni_patient.sort_values(by=['date'])
        uni_patient = uni_patient.iloc[-1,:]
        uni_patient = pd.DataFrame(uni_patient).T
        route.append(uni_patient)
    return pd.concat(route)

In [4]:
def minus_num(num):
    route = []
    for pat in patient['patient_id'].unique():
        uni_patient = patient[patient['patient_id'] == pat]
        uni_patient = uni_patient.sort_values(by=['date'])
        each_day = uni_patient.iloc[-1,:]['date']
        poi = uni_patient[(uni_patient['date']<=each_day)&(each_day-pd.Timedelta(days=num)<=uni_patient['date'])]
        route.append(poi)
    return pd.concat(route)

In [5]:
date_match = pd.DataFrame(range(1, 103), pd.date_range(start="2020-01-20",end="2020-04-30")).reset_index()
date_match.columns = ['date','num']
date_match['date'] = date_match['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [9]:
import geopandas as gpd
import requests
from shapely.geometry import Point
dong = gpd.read_file('Seoul_Gu25\Seoul_ADM_Final.shp')
gu = gpd.read_file('Seoul_Gu25\Seoul_Gu25.shp')

In [10]:
dong = dong.to_crs(epsg=4326)
gu = gu.to_crs(epsg=4326)
dong = dong[['ADM_DR_CD','geometry']]
dong.columns = ['dong_code','geometry']
gu = gu[['SIG_CD','SIG_ENG_NM','geometry']]
gu.columns = ['gu_code','gu','geometry']

In [11]:
def point_geom(p):
    geom = p.apply(lambda x : Point([x['longitude'],x['latitude']]), axis=1)
    p1 = gpd.GeoDataFrame(p, geometry=geom) #geom is a Series
    p1.crs = {'init' :'epsg:4326'}
    return p1

In [12]:
dong['center'] = dong.apply(lambda x: x['geometry'].centroid, axis=1)
dong['longitude'] = dong.apply(lambda x: x['center'].x, axis=1)
dong['latitude'] = dong.apply(lambda x: x['center'].y, axis=1)
gu['center'] = gu.apply(lambda x: x['geometry'].centroid, axis=1)
gu['longitude'] = gu.apply(lambda x: x['center'].x, axis=1)
gu['latitude'] = gu.apply(lambda x: x['center'].y, axis=1)

In [23]:
last_location = last_loc(patient)
minum_0 = minus_num(0)
minum_1 = minus_num(1)
minum_2 = minus_num(2)
minum_3 = minus_num(3)

In [24]:
last_location = point_geom(last_location)
minum_0 = point_geom(minum_0)
minum_1 = point_geom(minum_1)
minum_2 = point_geom(minum_2)
minum_3 = point_geom(minum_3)

In [25]:
def match(p, shp1, shp2):
    merge_dong = gpd.sjoin(p, shp1, op='within')[['patient_id','date','longitude_right','latitude_right']]
    merge_gu = gpd.sjoin(p, shp2, op='within')[['patient_id','date','longitude_right','latitude_right']]
    merge_dong.columns = ['patient_id','date','longitude','latitude']
    merge_gu.columns = ['patient_id','date','longitude','latitude']
    return merge_dong, merge_gu

In [26]:
last_location_dong, last_location_gu = match(last_location, dong, gu)
minum_0_dong, minum_0_gu = match(minum_0, dong, gu)
minum_1_dong, minum_1_gu = match(minum_1, dong, gu)
minum_2_dong, minum_2_gu = match(minum_2, dong, gu)
minum_3_dong, minum_3_gu = match(minum_3, dong, gu)

C:\Users\Main\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)
C:\Users\Main\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)
C:\Users\Main\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)
C:\Users\Main\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)
C:\Users\Main\Anaconda3\lib\site-packages\geopandas\

In [27]:
from scipy.spatial.distance import pdist, squareform
import geopy.distance
def dist_metric(coord1, coord2):
    coords_1 = (coord1[0], coord1[1])
    coords_2 = (coord2[0], coord2[1])
    return geopy.distance.vincenty(coords_1, coords_2).km

In [53]:
def dist_dataframe(p):
    pdis = []
    for i, dat in enumerate(p['date'].unique()):
        route1 = p[p['date'] == dat]
        tmp = pdist(route1[['latitude', 'longitude']].values, dist_metric)
        if len(tmp)>0:
            maximum = np.max(tmp)
            avg = np.mean(tmp)
            minimum = np.min(tmp)
            stdev = np.std(tmp)
            num_pts = len(route1)
            length = len(route1['patient_id'].unique())
            pdis.append([pd.to_datetime(dat).strftime('%Y-%m-%d'), avg, minimum, maximum, stdev, num_pts, length])
    pair = pd.DataFrame(pdis)
    pair.columns = ['date','average','minimum','maximum','stdev','num_pts','num_patient']
    pair = pair.merge(date_match, on='date', how='right').sort_values(by=['date'], axis=0).reset_index().replace(np.nan, 0)
    pair = pair[['date','average','minimum','maximum','stdev','num_pts','num_patient','num']]
    pair.columns = ['date','average_distance','minimum_distance','maximum_distance','standard deviation','num_pts','num_patient','day']
    pair1 = proximity_seoul[['day','d']]
    pair1[['num_pts','num_patient']] = pair[['num_pts','num_patient']]
    pair1[['average_distance','minimum_distance','maximum_distance','standard deviation']] = pair[['average_distance','minimum_distance','maximum_distance','standard deviation']]
    return pair1

In [29]:
proximity_seoul = pd.read_csv('proximity_seoul.csv')

In [30]:
ppair

,Unnamed: 0,day,d,num_pts,num_patient,average,minimum,maximum,stdev
0,0,1,Mon,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,Tue,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,Wed,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,Thu,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,Fri,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
import numpy as np
last_seoul = dist_dataframe(last_location)
last_minus_0 = dist_dataframe(minum_0)
last_minus_1 = dist_dataframe(minum_1)
last_minus_2 = dist_dataframe(minum_2)
last_minus_3 = dist_dataframe(minum_3)
last_seoul_dong = dist_dataframe(last_location_dong)
last_minus_0_dong = dist_dataframe(minum_0_dong)
last_minus_1_dong = dist_dataframe(minum_1_dong)
last_minus_2_dong = dist_dataframe(minum_2_dong)
last_minus_3_dong = dist_dataframe(minum_3_dong)
last_seoul_gu = dist_dataframe(last_location_gu)
last_minus_0_gu = dist_dataframe(minum_0_gu)
last_minus_1_gu = dist_dataframe(minum_1_gu)
last_minus_2_gu = dist_dataframe(minum_2_gu)
last_minus_3_gu = dist_dataframe(minum_3_gu)

C:\Users\Main\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [55]:
last_seoul[:100].to_csv('proximity_Seoulcity_delay0_last.csv', index=False)
last_minus_0[:100].to_csv('proximity_Seoulcity_delay0.csv', index=False)
last_minus_1[:100].to_csv('proximity_Seoulcity_delay1.csv', index=False)
last_minus_2[:100].to_csv('proximity_Seoulcity_delay2.csv', index=False)
last_minus_3[:100].to_csv('proximity_Seoulcity_delay3.csv', index=False)
last_seoul_dong[:100].to_csv('proximity_Seoulcity_dong_delay0_last.csv', index=False)
last_minus_0_dong[:100].to_csv('proximity_Seoulcity_dong_delay0.csv', index=False)
last_minus_1_dong[:100].to_csv('proximity_Seoulcity_dong_delay1.csv', index=False)
last_minus_2_dong[:100].to_csv('proximity_Seoulcity_dong_delay2.csv', index=False)
last_minus_3_dong[:100].to_csv('proximity_Seoulcity_dong_delay3.csv', index=False)
last_seoul_gu[:100].to_csv('proximity_Seoulcity_gu_delay0_last.csv', index=False)
last_minus_0_gu[:100].to_csv('proximity_Seoulcity_gu_delay0.csv', index=False)
last_minus_1_gu[:100].to_csv('proximity_Seoulcity_gu_delay1.csv', index=False)
last_minus_2_gu[:100].to_csv('proximity_Seoulcity_gu_delay2.csv', index=False)
last_minus_3_gu[:100].to_csv('proximity_Seoulcity_gu_delay3.csv', index=False)

In [47]:
last_seoul[:100]

,day,d,num_pts,num_patient,average_distance(km),minimum_distance(km),maximum_distance(km),standard deviation(km)
0,1,Mon,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,2,Tue,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,3,Wed,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,4,Thu,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,5,Fri,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
95,96,Fri,0.0,0.0,0.000000,0.000000,0.000000,0.0
96,97,Sat,0.0,0.0,0.000000,0.000000,0.000000,0.0
97,98,Sun,2.0,2.0,2.010412,2.010412,2.010412,0.0
98,99,Mon,0.0,0.0,0.000000,0.000000,0.000000,0.0
